In [ ]:
# Use logging to maintain more detailed information for reproducibility 
import logging

def tfLog():
  logging.basicConfig(level=logging.DEBUG, filename='myapp.log',
                      format='%(asctime)s %(levelname)s:%(message)s')
  try:
    logging.debug('######################################')
    logging.debug('Config Settings')
    logging.debug('######################################')
    logging.debug("Bucket:%s",BUCKET) 
    logging.debug("Folder:%s",FOLDER)
    logging.debug('Training base:%s',TRAINING_BASE)
    logging.debug('Eaval base:%s',EVAL_BASE) 
    logging.debug('Band order:%s',BANDS)
    logging.debug('Response:%s',RESPONSE)
    logging.debug('Features:%s',FEATURES)
    logging.debug('Kernal size:%d',KERNEL_SIZE)
    logging.debug('FEATURES_DICT:%s',FEATURES_DICT)
    logging.debug('Training size:%d',TRAIN_SIZE)
    logging.debug('Eval size:%d',EVAL_SIZE)
    logging.debug('batch size:%d',BATCH_SIZE)
    logging.debug('Epochs:%d',EPOCHS)
    logging.debug('Buffer size:%d',BUFFER_SIZE) 
    logging.debug('Optimizer:%s',OPTIMIZER) 
    # logging.debug('Loss:',LOSS)
    # logging.debug('Other metrics:',METRICS)
  except Exception as e:
    print('logging failed')
    print(e.args)

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Earth Engine install to notebook VM.
!pip install earthengine-api

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Tensorflow setup.
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()


In [ ]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'ee-tf'

In [ ]:
# normalize inputs
def basicNorm(img):
  img = img.toFloat();
  imgStd = img.reduceRegion(reducer=ee.Reducer.stdDev(),scale=0.5,maxPixels=1e13);
  imgMinMax = img.reduceRegion(reducer=ee.Reducer.minMax(),scale=0.5,maxPixels=1e13);
  
  imgMin = ee.Image.constant(imgMinMax.get(imgMinMax.keys().get(1)));
  imgMax = ee.Image.constant(imgMinMax.get(imgMinMax.keys().get(0)));
  
  normImg = img.subtract(imgMin).divide(imgMax.subtract(imgMin));
  return normImg.toFloat()


In [ ]:
from tensorflow.python.keras import backend

def dice_coeff(y_true, y_pred, smooth=1):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

## Set other global variables

In [ ]:
from tensorflow.python.keras import metrics
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'tahoe-ogfw-03112020-array-256' #good model'tahoe-ogfw-02292020'
TRAINING_BASE = 'Training_tahoe'
EVAL_BASE = 'Eval_tahoe'

# Specify inputs (Landsat bands) to the model and the response variable.

BANDS = ['R','G','B','NIR','L','O','ND']
RESPONSE = 'class'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 1
EPOCHS = 1
BUFFER_SIZE = 2000
OPTIMIZER = 'Adam'
LOSS = dice_loss
METRICS = [metrics.get('RootMeanSquaredError'),
    metrics.get('MeanAbsoluteError'),
    metrics.get('Accuracy'),
    dice_coeff,]

In [ ]:
tfLog()

In [ ]:
!ls
!cat myapp.log

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.gfile.Glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

# print(iter(training.take(1)).next())

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

# Model

Here we use the Keras implementation of the U-Net model as found [in the TensorFlow examples](https://github.com/tensorflow/models/blob/master/samples/outreach/blogs/segmentation_blogpost/image_segmentation.ipynb).  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.  Since impervious surface fraction is constrained to [0,1], with many values close to zero or one, a saturating activation function is suitable here.

In [ ]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [ ]:
m = get_model()
MODEL_FOLDER = 'test'
m.fit(
    x=training, 
    epochs=EPOCHS, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=evaluation,
    validation_steps=EVAL_SIZE)

modelDir = 'gs://{}/{}/{}'.format(BUCKET,FOLDER,MODEL_FOLDER)

tf.contrib.saved_model.save_keras_model(m, modelDir)
# TODO: add something to move log to model folder
tfLog()

Note that the notebook VM is sometimes not heavy-duty enough to get through a whole training job, especially if you have a large buffer size or a large number of epochs.  You can still use this notebook for training, but may need to set up an alternative VM ([learn more](https://research.google.com/colaboratory/local-runtimes.html)) for production use.  Alternatively, you can package your code for running large training jobs on Google's AI Platform [as described here](https://cloud.google.com/ml-engine/docs/tensorflow/trainer-considerations).  The following code loads a pre-trained model, which you can use for predictions right away.

In [ ]:
from tensorflow.python.tools import saved_model_utils
modelDir = 'gs://{}/{}/model-dice-256'.format('ee-tf',FOLDER)

meta_graph_def = saved_model_utils.get_meta_graph_def(modelDir, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "class"}) + "'"

# Put the EEified model next to the trained model directory.
# TODO: add eeidied dir, project into to log, add output name
EEIFIED_DIR = '{}/eeified'.format(modelDir)
PROJECT = 'tf-workshop-253517'
print(input_dict,output_dict)
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {modelDir} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

In [ ]:
import time 
MODEL_NAME = 'eeified_03112020_dice_256'
# PROJECT = 'tf-workshop-253517'
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --runtime-version=1.14 \
  --framework "TENSORFLOW" \
  --python-version=3.5

In [ ]:
# Load a trained model. 
MODEL_DIR = 'gs://ee-tf/tahoe-ogfw-02292020/model-ogwf-256'
m = tf.contrib.saved_model.load_keras_model(MODEL_DIR)
help(m.summary())
